# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
import os

home = os.path.expanduser("~")

db_name=  'test_1' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['lfw'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=['pose']  # 'gender', 'age', 'emotion', 'race', 'pose', 'angles'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

[16:52:14] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[16:52:14] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.8/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[16:52:15] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:52:15] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:52:15] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:52:15] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:52:15] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:52:15] ../src/nnvm/lega

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2023-03-19 16:52:16.179367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'test_1',
 'input_dir': '/home/jmacarulla/video_resources/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/video_resources/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f2b141224c0>,
 'databases': [<sql_face.databases.LFW at 0x7f2b141224f0>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['QMagFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
# import pandas as pd
# from tqdm import tqdm
# from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage

# session = database.session

# # generamos un datafame con la información que necesitamos para los registros sin quality
# # hacemos una query para actualizar los registros que no tienen quality


# df_a = pd.read_sql_query(session.query(QualityImage,FaceImage.embeddingModel_id,FaceImage.croppedImage_id)
#     .join(FaceImage)
#     .filter(QualityImage.quality != None,FaceImage.embeddingModel_id == 1).statement, con=session.bind)


# sql_b = (session.query(QualityImage, FaceImage)
#     .join(FaceImage)
#     .filter(QualityImage.quality == None).all())
# n1 = 100
# n = n1
# for Q,F in tqdm(sql_b):
#     #recorremos el resultado y actualizamos el objeto QualityImage


#     qm = Q.qualityModel_id
#     em = F.embeddingModel_id
#     ci = F.croppedImage_id

#     df_r = df_a.loc[(df_a['qualityModel_id'] == qm) & (df_a['croppedImage_id'] == ci) & (df_a['embeddingModel_id']== 1)]
#     assert len(df_r) == 1
#     Q.quality = float(df_r.quality)
#     if not n:
#         session.commit()
#         n = n1
#     else:
#         n-=1
# session.commit()



In [7]:
#database.create_tables(serfiq=serfiq)

In [8]:
database.update_tables(attributes_to_update, force_update = False, serfiq = serfiq)

Update pose: 100%|██████████| 29/29 [02:22<00:00,  4.91s/it]  
Update cropped images: 0it [00:00, ?it/s]
Computing embeddings DeepFace general: 0it [00:00, ?it/s]
Computing embeddings DeepFace mtcnn-serfiq: 0it [00:00, ?it/s]


SystemExit: => No checkpoint found at 'models/qmagface/magface_epoch_00025.pth'